In [9]:
import psycopg2
import facebook
import pandas as pd

In [2]:
access_token=#token from facebook developers
graph = facebook.GraphAPI(access_token, version='2.2')

In [3]:
# UF Ride group id: 154188317997936
feeds = graph.get_object(id='154188317997936?fields=feed{created_time,id,from,message}')['feed']

In [61]:
feeds_array=[]
for feed in feeds['data']:
    feeds_array.append([feed['created_time'],feed['message'].replace('\n', '')+"\n"])

In [62]:
def is_driving(message):
    driving=0
    if message.lower().split(' ')[0] in ["driving","heading", "leaving","departing"]:
        driving=1
    return driving   

In [77]:
def location(message,origin_destination):
    tmp, keyword, location = message.partition(origin_destination)
    return location


In [75]:
df=pd.DataFrame(feeds_array,columns=('message time raw','message raw'))
df['message time']=df['message time raw'].str[1:19]
df['message']=df['message raw'].str.replace("Hey ","").replace("Hi ","").replace("I am ","").replace("I'm ","")

df['driving']=df.message.apply(lambda x: "driving" if is_driving(x)==1 else "")
df['origin_destination']=df.message.apply(lambda x: location(x,"from"))

df[['message time', 'message', 'driving', 'origin_destination']]

,message time,message,driving,origin_destination
0,016-10-27T15:49:10,Anyone driving to West Palm Beach/South Florid...,,
1,016-10-27T15:37:06,Heading to Broward TODAY around 4PM!\n,driving,
2,016-10-27T15:20:21,Please tag if you know someone that can help: ...,,north miami beach to gainesville today... tex...
3,016-10-27T04:03:05,Driving from Gainesville to WPB this Friday @ ...,driving,Gainesville to WPB this Friday @ 1:00 msg me\n
4,016-10-27T03:29:03,"space for two, heading to wpb this friday at 6...",,
5,016-10-27T14:37:08,Driving SOUTH to Hollywood TODAY around 3pm. ...,driving,
6,016-10-27T00:54:33,Driving this Friday from Gainesville to Miami ...,driving,Gainesville to Miami (Aventura Area) around 4...
7,016-10-27T00:08:55,Looking for a ride from Gainesville to Sarasot...,,Gainesville to Sarasota/Tampa tomorrow or Fri...
8,016-10-27T13:45:15,Leaving to west palm beach today Thursday 27th...,driving,
9,016-10-27T00:04:35,Driving to tally on Friday and coming back Sun...,driving,


In [5]:
#"""CREATE TABLE posts(feed_id varchar(200),time varchar(200),message varchar(200))""")
try:
    conn = psycopg2.connect("dbname='ride' user='postgres' host='localhost' password=''")
except:
    print "I am unable to connect to the database"

In [6]:
cur = conn.cursor()

In [7]:
for feed in feeds['data']:
    feed_id = feed['id']
    time = feed['created_time']
    message = feed['message']

    query =  "INSERT INTO posts (feed_id, time, message) VALUES (%s, %s, %s);"
    data = (feed_id, time, message)

    cur.execute(query, data)
conn.commit()

In [78]:
cur.execute("""SELECT * from posts""")
rows = cur.fetchall()